In [438]:
import pandas
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import svm
from sklearn import preprocessing
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
import pickle
from sklearn import metrics
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import (RandomTreesEmbedding, IsolationForest, RandomForestClassifier, GradientBoostingClassifier)

def replaceNan(data, item):
    return [x if x is not None else item for x in data]

def getDenseFeatures(dataFrame, names, dictionary):
    X_dense = np.empty((len(dataFrame), 0), float)
    for name in names:
        values = dataFrame[name].values
        X_dense = np.column_stack([X_dense, replaceNan([dictionary[name].get(value) for value in values], 0.5)])
    return X_dense
def addNewFeature(data, vectors):
    return np.column_stack([data, vectors])
def getNewForIndexes(dataFrame, names):
    X_selectedFeature = np.empty((len(dataFrame), 0), float)
    for name in names:
        X_selectedFeature = np.column_stack([X_selectedFeature, dataFrame[name].values])
    return X_selectedFeature
def dataScaling(data, scaler):
    data = data/(data+1)
    return scaler.fit_transform(data)

def _computeFractionForAttribute(dataFrame, name):
    dictionary = dict()
    values = set(dataFrame[name].values)
    for value in values:
        ind = dataFrame[name].values==value
        subset = dataFrame['decision'][ind].values
        fraction = 1.0*len(filter(lambda x: x==1, subset)) / len(subset)
        dictionary[value] = fraction 
    return dictionary
def computeFractionsForAll(dataFrame, names):
    changeDict = dict()
    for name in names:
        changeDict[name] = _computeFractionForAttribute(dataFrame, name)
    return changeDict
def dropColumnsFromDataFrame(df, columns):
    for column in columns:
        df = df.drop(column, 1)
    return df

def writeToFile(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            f.write("%s\n" % item)
    f.close()
def printify(t):
    for x in t:
        print(x)
dataPrefix = '/home/data/aaia17/'
def readDataAsFrame(fNames):
    frame = []
    for fileName in fNames:
        fData = dataPrefix+fileName
        frame.append(pandas.read_csv(fData))
    return pandas.concat(frame)

def read_sparse_binary_set(matrix_path):
    return np.load(matrix_path)

def dataScaling(data, scaler):
    return scaler.fit_transform(data)

trainingFiles = ['trainingData_tabular_chunk1.csv', 'trainingData_tabular_chunk2.csv',
                 'trainingData_tabular_chunk3.csv', 'trainingData_tabular_chunk4.csv']
rawTrainingDataset = readDataAsFrame(trainingFiles)
testFiles = ['testData_tabular.csv']
rawTestDataset = readDataAsFrame(testFiles)

training_sparse = read_sparse_binary_set(dataPrefix + '/m.npy')
test_sparse = read_sparse_binary_set(dataPrefix + '/m_test.npy')


# Make dense features

In [439]:
columnsToDensify = ['opponent.hero_card_id', 'player.hero_card_id']
chd = computeFractionsForAll(rawTrainingDataset, columnsToDensify)
X_train_denseFeatures = getDenseFeatures(rawTrainingDataset, columnsToDensify, chd)
X_test_denseFeatures  = getDenseFeatures(rawTestDataset, columnsToDensify, chd)

#  Drop categorical or unnecessary features

In [440]:
YRawTrainingDataset = rawTrainingDataset.values[:,1]
columnsToDrop = ['decision','gamestate_id', 'opponent.hero_card_id', 'player.hero_card_id']
rawDroppedTrainingDataset = dropColumnsFromDataFrame(rawTrainingDataset, columnsToDrop)
rawDroppedTestDataset = dropColumnsFromDataFrame(rawTestDataset, columnsToDrop)

# Add some features before scaling

In [441]:
XRawTrainingDataset = rawDroppedTrainingDataset.values.astype(float)
XRawTestDataset     = rawDroppedTestDataset.values.astype(float)

XRawTrainingDataset = np.concatenate((XRawTrainingDataset, training_sparse[:,0:78]), axis = 1)
XRawTestDataset = np.concatenate((XRawTestDataset, test_sparse[:,0:78]), axis = 1)

assert np.all(training_sparse[:,78] == rawTrainingDataset.values[:,0]) and  np.all(test_sparse[:,78] == rawTestDataset.values[:,0])

# Data scaling

In [446]:
min_max_scaler = preprocessing.MinMaxScaler()
XTrainingDatasetScaledWithOutliers = dataScaling(XRawTrainingDataset, min_max_scaler)
XTestDatasetScaled = dataScaling(XRawTestDataset, min_max_scaler)

# Add new features after scaling

In [448]:
XTrainingDatasetScaledWithOutliers = addNewFeature(XTrainingDatasetScaledWithOutliers, X_train_denseFeatures)
XTestDatasetScaled = addNewFeature(XTestDatasetScaled, X_test_denseFeatures)

In [449]:
%%time
clf = GradientBoostingClassifier(n_estimators=450)
clf = clf.fit(XTrainingDatasetScaledWithOutliers, YRawTrainingDataset)
predictedTestWon = clf.predict_proba(XTestDatasetScaled)[:,1]
writeToFile(predictedTestWon, dataPrefix+'predicted41.txt')

CPU times: user 3h 39min 4s, sys: 1min 32s, total: 3h 40min 36s
Wall time: 3h 40min 55s
